In [1]:
import pandas as pd

# Analisis de trayectorio

Dataset de la informacion de paraderos (busstop.csv):
- Muestra informacion de los paraderos que se utilizan durante el trayectorio de los autobuses, tanto como la latitud y longitud.
- Se consideran puntos gps en formato de latitud y longitud


Se procesa la nueva trayectoria de paraderos y analizar los radios que se les asignaran para el geofencing, creando un nuevo archivo de la trayectoria de paraderos.

In [2]:
ida = pd.read_csv('../data/busstops_ida.csv', sep=',')
ida

,number,id,name,navigation,latitude,longitude,radio
0,0,0,inicio,159,-13.554978,-71.856025,30
1,1,36,primer stop,304,-13.554812,-71.857300,18
2,2,37,segundo stop,281,-13.553294,-71.861944,13
3,3,38,Ccollana,283,-13.552296,-71.867299,9
4,4,39,Puente Huaccoto,283,-13.551592,-71.870211,10
...,...,...,...,...,...,...,...
79,79,114,Segundo,221,-13.547528,-71.985248,10
80,80,115,Reservorio,225,-13.549660,-71.988003,10
81,81,116,Cuarto,230,-13.550408,-71.988880,10
82,82,117,Quinto,203,-13.550841,-71.989448,10


In [3]:
vuelta =  pd.read_csv('../data/busstop.csv', sep=',')


new_col = [0 for _ in range(vuelta.shape[0])]
vuelta['radio'] = new_col
vuelta = vuelta.reindex(columns=ida.columns)
vuelta = vuelta.tail(152-81)

In [4]:
df_final = pd.concat([ida, vuelta], ignore_index=True)
#df_final.to_csv('busstop_radio0.csv', sep=',')
df_final.drop(columns=['radio'], inplace=True)
df_final

,number,id,name,navigation,latitude,longitude
0,0,0,inicio,159,-13.554978,-71.856025
1,1,36,primer stop,304,-13.554812,-71.857300
2,2,37,segundo stop,281,-13.553294,-71.861944
3,3,38,Ccollana,283,-13.552296,-71.867299
4,4,39,Puente Huaccoto,283,-13.551592,-71.870211
...,...,...,...,...,...,...
150,146,183,Tapia,107,-13.549241,-71.878975
151,147,184,Forestal,107,-13.549985,-71.876345
152,148,185,Kayra,107,-13.550608,-71.874402
153,149,186,Puente,105,-13.551644,-71.870166


In [5]:
# Guarda el DataFrame modificado en un nuevo archivo CSV
df_final.to_csv('../data/busstop_radio0.csv', index=False)

In [6]:
# cargar datos en bruto
busstop_df = pd.read_csv('../data/busstop.csv', sep=',')
busstop_df


,number,id,name,latitude,longitude,navigation
0,0,38,Ccollana,-13.552296,-71.867299,283
1,1,39,Puente Huaccoto,-13.551592,-71.870211,283
2,2,40,Kayra,-13.550640,-71.873952,289
3,3,41,Forestal,-13.550020,-71.876104,286
4,4,42,Grifo Tapia,-13.549282,-71.878649,284
...,...,...,...,...,...,...
146,146,183,Tapia,-13.549241,-71.878975,107
147,147,184,Forestal,-13.549985,-71.876345,107
148,148,185,Kayra,-13.550608,-71.874402,107
149,149,186,Puente,-13.551644,-71.870166,105


Se crea un nuevo formato para el manejo de paraderos de la ruta, uniendo las columnas de longitud y latitud

In [7]:
# leer dataset busstop
busstopdf = pd.read_csv('../data/busstop_radio0.csv')

# diseño del df: creacion de latitude_longitude
busstopdf['latitude_longitude'] = busstopdf.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
busstopdf.drop(columns=['number', 'latitude', 'longitude'], inplace=True)

# para fines practicos modificamos el id = index para mejor lectura
busstopdf['id'] = [f'{i+100}' for i in busstopdf.index]

# mostrar info y elementos
print(busstopdf.info())
busstopdf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  155 non-null    object
 1   name                155 non-null    object
 2   navigation          155 non-null    int64 
 3   latitude_longitude  155 non-null    object
dtypes: int64(1), object(3)
memory usage: 5.0+ KB
None


,id,name,navigation,latitude_longitude
0,100,inicio,159,"(-13.5549784, -71.856025)"
1,101,primer stop,304,"(-13.5548123, -71.8573001)"
2,102,segundo stop,281,"(-13.5532944, -71.8619442)"
3,103,Ccollana,283,"(-13.552296, -71.867299)"
4,104,Puente Huaccoto,283,"(-13.551592, -71.870211)"
...,...,...,...,...
150,250,Tapia,107,"(-13.549241, -71.878975)"
151,251,Forestal,107,"(-13.549985, -71.876345)"
152,252,Kayra,107,"(-13.550608, -71.874402)"
153,253,Puente,105,"(-13.551644, -71.870166)"


Generar los links de ida y vuelta

In [8]:
# creamos dataframe links y renombramos columnas
tmp1 = busstopdf[['id','name']].head(busstopdf.shape[0]-1).reset_index(drop=True)
# tmp1.rename(columns={'id':'ini_id', 'name':'ini_name'}, inplace=True)
tmp1.rename(columns={'id':'init_stop', 'name':'init_name'}, inplace=True)

tmp2 = busstopdf[['id','name']].tail(busstopdf.shape[0]-1).reset_index(drop=True)
# tmp2.rename(columns={'id':'end_id', 'name':'end_name'}, inplace=True)
tmp2.rename(columns={'id':'end_stop', 'name':'end_name'}, inplace=True)
linksdf = pd.concat([tmp1, tmp2], axis=1)

# creamos la columnas link
# linksdf['link'] = linksdf.apply(lambda row: f"{row['ini_id']}:{row['end_id']}", axis=1)
linksdf['link'] = linksdf.apply(lambda row: f"{row['init_stop']}:{row['end_stop']}", axis=1)
linksdf['id_link'] = linksdf.index
linksdf.insert(0, 'id_link', linksdf.pop('id_link'))
linksdf

,id_link,init_stop,init_name,end_stop,end_name,link
0,0,100,inicio,101,primer stop,100:101
1,1,101,primer stop,102,segundo stop,101:102
2,2,102,segundo stop,103,Ccollana,102:103
3,3,103,Ccollana,104,Puente Huaccoto,103:104
4,4,104,Puente Huaccoto,105,Kayra,104:105
...,...,...,...,...,...,...
149,149,249,Posta,250,Tapia,249:250
150,150,250,Tapia,251,Forestal,250:251
151,151,251,Forestal,252,Kayra,251:252
152,152,252,Kayra,253,Puente,252:253


a
<img src='graficos/trayectoria_bus_stops.png' width='600' height='300'>

# 2. Analisis de los registros de monitorizacion

Tareas a realizar:
- Filtrar los paraderos de vuelta (los que se usa y eliminar los que no se usan y que conincidan con el de ida)

Se da nuevo formato a la tabla, creando nuevas columnas y eliminando otras.

In [10]:
# cargar datos en bruto
monitoringdf = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')

# Creacion del las columnas datetime y latitude_longitude
monitoringdf['datetime'] = monitoringdf['date'] + ' ' + monitoringdf['time']
monitoringdf['latitude_longitude'] = monitoringdf.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# eliminar las columnas latitude, longitude, time, velocity. Luego reordenar columnas
monitoringdf.drop(columns=['latitude', 'longitude','time'], inplace=True)
monitoringdf = monitoringdf[['id', 'date','datetime', 'latitude_longitude', 'lap', 'navigation', 'vehicle_id']]
monitoringdf

,id,date,datetime,latitude_longitude,lap,navigation,vehicle_id
0,13173398,2023-03-01,2023-03-01 05:10:11,"(-13.5495973, -71.8775099)",1,285,SJ39
1,13177467,2023-03-01,2023-03-01 06:04:51,"(-13.5245292, -71.9775272)",1,298,SJ34
2,13189674,2023-03-01,2023-03-01 07:22:21,"(-13.5341828, -71.9807088)",1,7,SJ29
3,13192729,2023-03-01,2023-03-01 07:39:04,"(-13.5511464, -71.9869284)",1,202,SJ21
4,13193272,2023-03-01,2023-03-01 07:41:52,"(-13.548642, -71.88101)",1,107,SJ34
...,...,...,...,...,...,...,...
1685065,17241845,2023-03-25,2023-03-25 08:50:24,"(-13.5254398, -71.9750786)",2,288,SJ24
1685066,17253753,2023-03-25,2023-03-25 10:01:56,"(-13.5511209, -71.9896213)",2,253,SJ16
1685067,17257644,2023-03-25,2023-03-25 10:26:10,"(-13.5432069, -71.8977847)",2,126,SJ24
1685068,17266468,2023-03-25,2023-03-25 11:14:26,"(-13.5327737, -71.9218652)",1,258,SJ43


# Cleaning

Mostramos la cantidad de datos, existen puntos que por error de lectura se leen dos veces, estos puntos se detectan si en las columnas datetime y latitud-longitud existen datos iguales, esto no debe suceder ya que el bus esta en constante movimiento como el tiempo. Como consecuencia se decide eliminar estos duplicados.

In [11]:
# DUPLICADOS
print(f'Shape dataset: {monitoringdf.shape}')

# Verificar y contar si hay puntos duplicados segun datetime y latitude_longitude
base = ['datetime', 'latitude_longitude']
twocols_dupli = monitoringdf.duplicated(subset=base)
print(f"Nro de duplicados en {base}: {twocols_dupli.sum()}")

# ============================ Solo para comprobacion ============================
# Verificar y contar si hay puntos duplicados segun latitude_longitude
# onecol_dupli =  monitoringdf['latitude_longitude'].duplicated()
# print(len(twocols_dupli), len(onecol_dupli))
# print(twocols_dupli.sum(), onecol_dupli.sum())
# print(abs(twocols_dupli.sum() - onecol_dupli.sum()))
# tmp = [ not x==y for x,y in zip(twocols_dupli, onecol_dupli)]
# data_tmp = monitoringdf.copy()
# data_tmp['dupli'] = tmp
# data_tmp[data_tmp['dupli'] == True]
# ================================================================================

# Eliminar columnas
monitoringdf.drop_duplicates(subset=base, inplace=True)
print(f"Duplicados eliminados => shape dataset: {monitoringdf.shape}")

Shape dataset: (1685070, 7)
Nro de duplicados en ['datetime', 'latitude_longitude']: 13116
Duplicados eliminados => shape dataset: (1671954, 7)


Comprobamos si existen casillas vacias (Nans) en todo el dataframe.

In [12]:
# NANs
# verificar si hay datos faltantes
print("Cantidad de NaNs:")
for col in monitoringdf:
    nans = monitoringdf[col].isna().sum()
    print(f"\tColumna {col}: {nans}")

Cantidad de NaNs:
	Columna id: 0
	Columna date: 0
	Columna datetime: 0
	Columna latitude_longitude: 0
	Columna lap: 0
	Columna navigation: 0
	Columna vehicle_id: 0


# Funciones para calcular el travel time

In [46]:
from haversine import haversine as hvs, Unit
import numpy as np

# FUNCION: filtrar los puntos de ida
def idaPoints(df):
    end1_stop = {'latitude_longitude':(-13.5513636,-71.988304), 'navigation':75, 'radio':50}
    end2_stop = {'latitude_longitude':(-13.5511395,-71.986925), 'navigation':206, 'radio':50}
    for i, row in df.iterrows():
        if (itPassed(row, end1_stop) or itPassed(row, end2_stop)):
            break
    return df.head(i+1)

# FUNCION: ver si un punto paso por un stop
def itPassed(point, stop):
    nav = abs(point['navigation'] - stop['navigation'])
    dis = round(hvs(point['latitude_longitude'], stop['latitude_longitude'], unit=Unit.METERS), 2)
    return (nav<=45 and dis<=stop['radio'])

# FUNCION: 
def searchStopsPoints(row, stops, r_nav=45, r_dtec=60):
    # verificar la navegacion y el radio de distancia
    stops = stops[abs(stops['navigation'] - row['navigation']) <= r_nav]
    stops['dis'] = stops['latitude_longitude'].apply(lambda stop: round(hvs(row['latitude_longitude'], stop, unit=Unit.METERS), 2))
    stops = stops[stops['dis'] <= r_dtec]
    
    # preparar el output
    row['stop'] = str(stops.loc[stops['dis'].idxmin(), 'id']) if len(stops) != 0 else np.nan
    row['dis'] = float(stops.loc[stops['dis'].idxmin(), 'dis']) if len(stops) != 0 else 0    
    #row['long_lati'] = row['latitude_longitude']
    return row

# FUNCION: CALCULA EL TIEMPO EN SEG DE UN PUNTO CON SU ANTERIOR
def time_travel(row, df):
    if {row['init_stop'], row['end_stop']}.issubset(set(df.index)):
        # print('if')
        ti, te =  df.loc[row['init_stop'], 'datetime'], df.loc[row['end_stop'], 'datetime']
        row['date_time_init'] = ti
        row['date_time_end'] = te
        time = abs((ti - te).total_seconds())
        
        #row['time_travel'] =  time if time < 300 else np.nan
        row['time_travel'] =  time
        row['init_idp'], row['end_idp'] = df.loc[row['init_stop'], 'id'], df.loc[row['end_stop'], 'id']

        row['ini_lat'] = df.loc[row['init_stop'], 'latitude_longitude']
        row['end_lat'] = df.loc[row['end_stop'], 'latitude_longitude']

        row['veh'] = df.loc[row['init_stop'], 'vehicle_id']
        row['lap'] = df.loc[row['init_stop'], 'lap']
        row['date'] = df.loc[row['init_stop'], 'date']


    else:
        # print('else')
        row['date_time_init'] = np.nan
        row['date_time_end'] = np.nan
        row['time_travel'] = np.nan
        row['init_idp'] = np.nan
        row['end_idp'] = np.nan
        row['veh'] = np.nan
        row['lap'] = np.nan
        row['date'] = np.nan
        
    return row

# Crear secuencias y calcular tiempos de viaje

Convierte las fechas en formato datetime

In [16]:
# ordenar df por fechas
monitoringdf['datetime'] = pd.to_datetime(monitoringdf['datetime'], format='%Y-%m-%d %H:%M:%S')
monitoringdf = monitoringdf.sort_values(by='datetime')
print("Type columna 'datetime' : ", monitoringdf['datetime'].dtype)
monitoringdf.head()

Type columna 'datetime' :  datetime64[ns]


,id,date,datetime,latitude_longitude,lap,navigation,vehicle_id
29756,13172252,2023-03-01,2023-03-01 00:01:34,"(-13.5513563, -71.9902231)",5,140,SJ40
29311,13172234,2023-03-01,2023-03-01 00:01:49,"(-13.5506013, -71.8786173)",0,185,SJ10
29780,13172253,2023-03-01,2023-03-01 00:03:25,"(-13.5506231, -71.8786667)",0,206,SJ10
31234,13172309,2023-03-01,2023-03-01 00:05:13,"(-13.5383472, -71.9807985)",5,0,SJ08
29831,13172255,2023-03-01,2023-03-01 00:05:30,"(-13.55065, -71.8785454)",0,113,SJ10


# Crear dataset

In [26]:
# preparar el dataset para iterar en todos los posibles samples
monitoringdf_tmp = monitoringdf.copy()
#monitoringdf_tmp = monitoringdf_tmp[(monitoringdf_tmp['date'] == '2023-03-01')] #| (monitoringdf_tmp['date'] == '2023-03-02')]

# Crear columna sample para identificar los samples, utilizando la fecha vehiculo y lapso, todo ello encapsulado en un string
monitoringdf_tmp['sample'] = monitoringdf.apply(lambda row: f"{row['date']} {row['vehicle_id']} {row['lap']}" , axis=1)

# Contar la cantidad de secuencias posibles en base a los valores repetidos que existe en la columna sample
n_posibles_samples = len(monitoringdf_tmp['sample'].unique())
print('nro posibles samples:', n_posibles_samples)

nro posibles samples: 5015


In [48]:
# iterar en todos los posibles
n_descartados = 0
id_sample = 0
list_err = []
list_samplesdf = []
date = ''
bt = 0

#Se ordena los posibles samples de manera ascendente
for ids, pointsdf in monitoringdf_tmp.groupby('sample'):

    # Seguir procesamiento
    if date != pointsdf['date'].unique()[0]:
        if (bt == 1):
            break
        bt+=1

        date = pointsdf['date'].unique()[0]
        print(date)

    # ordenar la secuencia por datetime
    pointsdf.sort_values(by='datetime', inplace=True)

    # identificar los puntos de ida del sample
    #idaPointsdf = idaPoints(pointsdf.reset_index(drop=True))

    # verificar puntos por stop
    #stopsPointsdf = idaPointsdf.apply(lambda row: searchStopsPoints(row, busstopdf), axis=1)
    
    stopsPointsdf = pointsdf.apply(lambda row: searchStopsPoints(row, busstopdf), axis=1)

    stopsPointsdf = stopsPointsdf[stopsPointsdf['stop'].notna()].reset_index()

    # Seleccionar un unico punto por stops
    stopsPointsdf.sort_values(by=['stop', 'dis'], ascending=True, inplace=True, ignore_index=True, key=np.int64)
    stopsPointsdf.drop_duplicates(subset=['stop'], keep='first', inplace=True, ignore_index=True)

    # filtrar los stop sin puntos 
    missing_stops = list(set(busstopdf['id']) - set(stopsPointsdf['stop'].unique()))

    # verificar la cantidad de puntos filtrados
    if len(missing_stops) < 24:
        # verificar la secuencialidad de los datetime
        byS = list(stopsPointsdf.sort_values(by='stop')['stop'])
        byT = list(stopsPointsdf.sort_values(by='datetime')['stop'])
        if byS == byT:
            # crear sample y calcular el travel_time
            sampledf = linksdf.copy()
            stopsPointsdf.set_index('stop', inplace=True)
            sampledf = sampledf.apply(lambda row: time_travel(row, stopsPointsdf), axis=1)
            
            # crear el id_sample
            # sampledf['id_sample'] = 
            sampledf.insert(0, 'id_sample', [id_sample] * sampledf.shape[0])
            list_samplesdf.append(sampledf)
            id_sample += 1
        else:
            # registrar sample defectoso
            list_err.append(stopsPointsdf)
            print(' ', ids)
    else:
        n_descartados += 1

# mostrar nro de samples encontrados
print('='*100)
print(f"{len(list_samplesdf)} Samples buenos")
print(f"{len(list_err)} Samples con errores de continuidad de tiempo")
print(f"{n_descartados} Samples descartados por falta de puntos")

2023-03-01
  2023-03-01 SJ01 5
  2023-03-01 SJ04 3
  2023-03-01 SJ04 5
  2023-03-01 SJ05 4
  2023-03-01 SJ07 1
  2023-03-01 SJ07 5
  2023-03-01 SJ09 1
  2023-03-01 SJ09 3
  2023-03-01 SJ13 2
  2023-03-01 SJ14 5
  2023-03-01 SJ15 3
  2023-03-01 SJ15 4
  2023-03-01 SJ16 2
  2023-03-01 SJ16 5
  2023-03-01 SJ17 5
  2023-03-01 SJ19 3
  2023-03-01 SJ20 2
  2023-03-01 SJ21 2
  2023-03-01 SJ21 4
  2023-03-01 SJ21 5
  2023-03-01 SJ22 5
  2023-03-01 SJ23 4
  2023-03-01 SJ25 1
  2023-03-01 SJ26 1
  2023-03-01 SJ27 6
  2023-03-01 SJ28 3
  2023-03-01 SJ30 1
  2023-03-01 SJ30 2
  2023-03-01 SJ32 1
  2023-03-01 SJ33 2
  2023-03-01 SJ34 5
  2023-03-01 SJ35 3
  2023-03-01 SJ35 5
  2023-03-01 SJ37 2
  2023-03-01 SJ37 3
  2023-03-01 SJ37 5
  2023-03-01 SJ38 3
  2023-03-01 SJ38 5
  2023-03-01 SJ39 5
  2023-03-01 SJ40 5
  2023-03-01 SJ42 1
111 Samples buenos
41 Samples con errores de continuidad de tiempo
75 Samples descartados por falta de puntos


In [41]:
monitoringdf_tmp

,id,date,datetime,latitude_longitude,lap,navigation,vehicle_id,sample
29756,13172252,2023-03-01,2023-03-01 00:01:34,"(-13.5513563, -71.9902231)",5,140,SJ40,2023-03-01 SJ40 5
29311,13172234,2023-03-01,2023-03-01 00:01:49,"(-13.5506013, -71.8786173)",0,185,SJ10,2023-03-01 SJ10 0
29780,13172253,2023-03-01,2023-03-01 00:03:25,"(-13.5506231, -71.8786667)",0,206,SJ10,2023-03-01 SJ10 0
31234,13172309,2023-03-01,2023-03-01 00:05:13,"(-13.5383472, -71.9807985)",5,0,SJ08,2023-03-01 SJ08 5
29831,13172255,2023-03-01,2023-03-01 00:05:30,"(-13.55065, -71.8785454)",0,113,SJ10,2023-03-01 SJ10 0
...,...,...,...,...,...,...,...,...
1585942,17270531,2023-03-25,2023-03-25 11:37:16,"(-13.550582, -71.8741605)",3,285,SJ20,2023-03-25 SJ20 3
1585989,17270532,2023-03-25,2023-03-25 11:37:16,"(-13.5504177, -71.9870624)",2,230,SJ03,2023-03-25 SJ03 2
1586202,17270537,2023-03-25,2023-03-25 11:37:17,"(-13.5217018, -71.9646513)",3,289,SJ32,2023-03-25 SJ32 3
1586071,17270534,2023-03-25,2023-03-25 11:37:17,"(-13.5405389, -71.9812735)",2,13,SJ04,2023-03-25 SJ04 2


In [51]:
list_samplesdf[0].columns

Index(['id_sample', 'date', 'date_time_end', 'date_time_init', 'end_idp',
       'end_lat', 'end_name', 'end_stop', 'id_link', 'ini_lat', 'init_idp',
       'init_name', 'init_stop', 'lap', 'link', 'time_travel', 'veh'],
      dtype='object')

In [52]:
columnas_ord = ['id_sample', 'date', 'date_time_end', 'date_time_init', 'end_idp', 'end_name', 'end_stop', 'id_link', 'init_idp',
       'init_name', 'init_stop', 'lap', 'link', 'time_travel', 'veh', 'ini_lat',
       'end_lat']

In [55]:
list_samplesdf[109][columnas_ord]

,id_sample,date,date_time_end,date_time_init,end_idp,end_name,end_stop,id_link,init_idp,init_name,init_stop,lap,link,time_travel,veh,ini_lat,end_lat
0,109,2023-03-01,2023-03-01 14:37:49,2023-03-01 14:04:39,13277666.0,primer stop,101,0,13271047.0,inicio,100,3.0,100:101,1990.0,SJ43,"(-13.5550828, -71.8559498)","(-13.5550275, -71.8570669)"
1,109,2023-03-01,2023-03-01 14:39:56,2023-03-01 14:37:49,13278087.0,segundo stop,102,1,13277666.0,primer stop,101,3.0,101:102,127.0,SJ43,"(-13.5550275, -71.8570669)","(-13.5532644, -71.8622568)"
2,109,2023-03-01,2023-03-01 14:41:46,2023-03-01 14:39:56,13278467.0,Ccollana,103,2,13278087.0,segundo stop,102,3.0,102:103,110.0,SJ43,"(-13.5532644, -71.8622568)","(-13.552325, -71.8672449)"
3,109,2023-03-01,2023-03-01 14:42:30,2023-03-01 14:41:46,13278612.0,Puente Huaccoto,104,3,13278467.0,Ccollana,103,3.0,103:104,44.0,SJ43,"(-13.552325, -71.8672449)","(-13.5516429, -71.8699255)"
4,109,2023-03-01,2023-03-01 14:43:59,2023-03-01 14:42:30,13278918.0,Kayra,105,4,13278612.0,Puente Huaccoto,104,3.0,104:105,89.0,SJ43,"(-13.5516429, -71.8699255)","(-13.5505811, -71.8741039)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,109,2023-03-01,2023-03-01 17:17:27,2023-03-01 17:17:15,13309590.0,Tapia,250,149,13309557.0,Posta,249,3.0,249:250,12.0,SJ43,"(-13.5490034, -71.8797405)","(-13.5492024, -71.8788978)"
150,109,2023-03-01,2023-03-01 17:18:16,2023-03-01 17:17:27,13309738.0,Forestal,251,150,13309590.0,Tapia,250,3.0,250:251,49.0,SJ43,"(-13.5492024, -71.8788978)","(-13.5499915, -71.8762344)"
151,109,2023-03-01,2023-03-01 17:18:40,2023-03-01 17:18:16,13309820.0,Kayra,252,151,13309738.0,Forestal,251,3.0,251:252,24.0,SJ43,"(-13.5499915, -71.8762344)","(-13.5505434, -71.8744531)"
152,109,2023-03-01,2023-03-01 17:20:18,2023-03-01 17:18:40,13310127.0,Puente,253,152,13309820.0,Kayra,252,3.0,252:253,98.0,SJ43,"(-13.5505434, -71.8744531)","(-13.5515743, -71.8703316)"


In [54]:
list_samplesdf[109][columnas_ord].to_csv('sample_ida_vuela.csv', index=False)